In [1]:
import os
import pandas as pd
import numpy as np

# sub_df = pd.read_csv('sub_data/all.csv')
vol_df = pd.read_csv('VolumeVoxel_df.csv')
# only CN and MCI spreadsheets have the image data id necessary to make the map
sub_df = pd.read_csv('sub_data/data_list_SCZ_AD_MCI_CN (1).xlsx - CN_All.csv').append(pd.read_csv('sub_data/data_list_SCZ_AD_MCI_CN (1).xlsx - MCI_All.csv')).append(pd.read_csv('sub_data/data_list_SCZ_AD_MCI_CN (1).xlsx - AD_All.csv'))

In [29]:
def pull_nth(n:int, df, id_col_name='filename'): #n can be -1
    trunc_ids = list(set(full_id[:-4] for full_id in df[id_col_name]))
    rows_to_return = []
    for trunc_id in trunc_ids:
        nth = sorted([ID for ID in df[id_col_name] if trunc_id in ID], key=lambda x:(x[-3], x[-1]))[n]
        rows_to_return.append(int(df[df[id_col_name] == nth].index.asi8))

    return df.iloc[rows_to_return]
        
        
res = pull_nth(0, vol_df.iloc[:423])

/home/adrien/MRI-classifier/shapenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Index.asi8 is deprecated and will be removed in a future version
  


In [30]:

res

,filename,Target,Angular Gyrus (Volume),Brain-Stem (Volume),Central Opercular Cortex (Volume),"Cingulate Gyrus, anterior division (Volume)","Cingulate Gyrus, posterior division (Volume)",Cuneal Cortex (Volume),Frontal Medial Cortex (Volume),Frontal Operculum Cortex (Volume),...,Superior Parietal Lobule (Voxel),"Superior Temporal Gyrus, anterior division (Voxel)","Superior Temporal Gyrus, posterior division (Voxel)",Supracalcarine Cortex (Voxel),"Supramarginal Gyrus, anterior division (Voxel)","Supramarginal Gyrus, posterior division (Voxel)","Temporal Fusiform Cortex, anterior division (Voxel)","Temporal Fusiform Cortex, posterior division (Voxel)",Temporal Occipital Fusiform Cortex (Voxel),Temporal Pole (Voxel)
106,023_S_0031_1_1,CN,4015.232860,19985.45874,6599.736662,12281.533040,6696.489261,2127.347771,1312.207124,1995.522355,...,3932.0,367.0,3439.0,73.0,3374.0,2395.0,290.0,2355.0,2210.0,13715.0
119,023_S_0061_1_1,CN,3945.311856,24966.84184,5939.711696,6901.275761,5881.727933,2233.582861,2550.077565,1124.643398,...,2835.0,337.0,3315.0,40.0,3018.0,3438.0,536.0,3095.0,2795.0,10185.0
182,023_S_1046_1_1,MCI,5001.572947,30996.33809,7781.422278,10584.027810,7364.624533,1449.210552,3294.858040,1376.151177,...,3911.0,472.0,4184.0,88.0,2795.0,3130.0,803.0,3019.0,2987.0,13834.0
373,136_S_0086_1_1,CN,3640.715665,17947.68525,6105.050853,10067.148810,6271.199436,2762.674979,2253.314359,1029.636109,...,2592.0,510.0,2802.0,75.0,2529.0,1717.0,1113.0,2406.0,2375.0,12386.0
94,018_S_0633_1_1,AD,2813.069545,22356.11584,6196.783389,14464.433720,6011.841100,1445.469991,1786.153154,2090.334550,...,3925.0,381.0,2875.0,67.0,2517.0,1321.0,796.0,3055.0,2203.0,19716.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,027_S_1387_1_1,MCI,4691.789244,16387.52581,7993.150962,10522.186820,14395.864890,2655.547893,2298.904437,826.545306,...,4611.0,208.0,2943.0,117.0,2007.0,2222.0,555.0,1665.0,2552.0,7229.0
347,127_S_0844_1_1,AD,4444.493813,18493.59243,6854.590228,9035.944724,4793.357186,2272.084531,2224.802682,1235.717515,...,3169.0,535.0,2528.0,59.0,2917.0,2196.0,668.0,2620.0,1829.0,11483.0
272,037_S_0501_1_1,MCI,6173.749145,19176.05725,6910.410185,7769.848066,5429.865937,2063.132391,1433.600162,2095.632143,...,4638.0,481.0,2914.0,62.0,3500.0,2910.0,417.0,2802.0,2258.0,10930.0
225,027_S_0835_1_1,MCI,5052.737564,23254.40502,7430.850927,9722.180628,7939.499603,1807.992926,3740.134700,1175.195402,...,3842.0,645.0,3630.0,69.0,2994.0,2232.0,823.0,2905.0,2563.0,15010.0


In [ ]:
# extract just the problematic portion
# sub_df = sub_df.iloc[448:]
vol_df = vol_df.iloc[:423]
vol_df

In [ ]:
sub_df.drop_duplicates(inplace=True)
vol_df.drop_duplicates(subset='filename', inplace=True)

In [ ]:
vol_df

In [ ]:
# now I need to somehow build a map between rows... oh god
# loop through set of the truncated ids
truncIDs = set(sub_df['subject'])
# truncIDs

In [ ]:
# all we care about in the vol df is the id
vol_ids = list(vol_df['filename'])

In [ ]:
sub_df[sub_df.subject == '136_S_1227'].to_numpy()[:,5]

In [ ]:
godmap = {}
flagged = []
c = 0
for truncID in truncIDs:
    print(truncID)
    sub_rows = sub_df[sub_df.subject == truncID]
    vids = [vid for vid in vol_ids if truncID in vid]
    if not len(sub_rows) == len(vids):
        print('WAT')
        c+=1
        continue
    # find coreespondence
    # first sort rows by visit
    sorted_subs = np.array(sorted(sub_rows.to_numpy(), key=lambda row: (row[5], row[0])))
    sorted_vids = sorted(vids, key=lambda vid: (vid[-3], vid[-1]))
    
    
    # if the second last digit is not one despite there being duplicate tweets... idk what to do.
    for i in range(len(vids)):
        godmap[vids[i]] = list(sorted_subs[i])
    if all([vid[-3] == '1' for vid in sorted_vids]):
        print('good.')
    else: # ...just sort by visit and hope it's fine?
        flagged = flagged + vids
            


    # break

In [ ]:
c

In [ ]:
import json
with open('id_map.json', 'w') as f:
    f.write(json.dumps(godmap))

In [ ]:
len(list(godmap.keys()))
# godmap.keys()
map_df = pd.DataFrame([[key, godmap[key], key in flagged] for key in godmap])
map_df.to_csv('id_map.csv')

In [ ]:
map_df

In [ ]:
raise Exception('stop')

In [ ]:
merged1 = pd.merge(sub_df, vol_df, how='left', on='filename')

In [ ]:
merged1

In [ ]:
# list(merged1['class'] == merged1['Target'])[:423]

In [ ]:
merged1.to_csv('merged1.csv')

In [ ]:
sub_id_col = vol_df['filename'].iloc[:423].apply(get_id)
vol_df['subject'] = sub_id_col

In [ ]:
vol_df

In [ ]:
sub_df

In [ ]:
merged2 = pd.merge(sub_df, vol_df, how='inner', on='subject')

In [ ]:
# list(merged2.columns)

In [ ]:
fff = merged2.drop_duplicates('filename_y')

In [ ]:
all(fff['class'] == fff['Target'])

In [ ]:
##########################

In [ ]:
worked = pd.merge(sub_df, vol_df_bottom, on='filename', how='left')

In [ ]:
worked[:448]

In [ ]:
# worked.to_csv('merged.csv')

In [ ]:
worked_top = worked.iloc[:448].copy(deep=True) #merge completed
worked_bottom = worked.iloc[448:].copy(deep=True)

In [ ]:
worked_bottom

In [ ]:
worked_bottom = pd.merge(worked_bottom, vol_df_top, how='outer', on='subject')

In [ ]:
worked_bottom

In [ ]:
worked = pd.merge(worked, vol_df_top, on='subject', how='left')

In [ ]:
worked.to_csv('merged.csv')